# Reuse of SGTPy phase equilibra and SGT functions

In SGTPy the functions included in phasepy for phase equilibria and SGT solution are implemented for the SAFT-VR-Mie. These functions could be reuse with other models implemented by other users.

In this notebook the needed methods and attributes that the ``eos`` should have implemented are described.

In [1]:
import numpy as np 
from SGTPy import component, mixture, saftvrmie

In [2]:
ethanol = component('ethanol2C', ms = 1.7728, sigma = 3.5592 , eps = 224.50,
                    lambda_r = 11.319, lambda_a = 6., eAB = 3018.05, rcAB = 0.3547,
                    rdAB = 0.4, sites = [1,0,1], cii= 5.3141080872882285e-20)

hexane = component('hexane', ms = 1.96720036, sigma = 4.54762477, eps = 377.60127994, 
                   lambda_r = 18.41193194, cii = 3.581510586936205e-19)

mix = mixture(hexane, ethanol)

# kij fitted to experimental data
kij = 0.011818492037463553
Kij = np.array([[0, kij], [kij, 0]])
mix.kij_saft(Kij)
eos = saftvrmie(mix)

The ``eos.nc`` attribute indicates the number of components in the mixture

In [3]:
eos.nc

2

The ``eos.secondordersgt`` attribute indicates whether the EoS includes the derivatives of the chemical potential
respect to the density vector through the ``eos.dmuad_aux`` method.

In [4]:
eos.secondordersgt

True

In [5]:
T = 320. #K
P = 1e5 # Pa
x = np.array([0.4, 0.6])

rho = 10996.38419604313 # mol/m3
rhoi = x*rho
RT = T*8.314

The ``eos.temperature_aux`` method computes all the temperature dependent parameters.

In [6]:
temp_aux = eos.temperature_aux(T)

The ``eos.logfugef_aux`` method computes the effective fugacity coefficients, volume root and non-bonded sites fractions.

Used in SGTPy for phase equilibria computation.

In [7]:
eos.logfugef_aux(x, temp_aux, P, 'L')

(array([ 0.07197118, -1.06735572]),
 9.093898341236875e-05,
 array([0.06372576, 0.25243962]))

The ``eos.muad_aux`` method computes the chemical potential and non-bonded sites fractions.

Used in SGTPy for density profiles calculations.

In [8]:
eos.muad_aux(rhoi, temp_aux)

(array([-0.35114761, -1.0850094 ]), array([0.06372576, 0.25243962]))

The ``eos.dmuad_aux`` method computes the chemical potential, it derivatives respect to the density vector and non-bonded sites fractions.

Used in SGTPy for density profiles calculations in the BVP problem (i.e. beta !=0)

In [9]:
eos.dmuad_aux(rhoi, temp_aux)

(array([-0.35114761, -1.0850094 ]),
 array([[0.00380566, 0.00166061],
        [0.00166061, 0.00073812]]),
 array([0.06372576, 0.25243962]))

The ``eos.a0ad_aux`` method computes the Helmholtz density energy and non-bonded sites fractions.

Used to compute the interfacial tension in SGT.

In [10]:
eos.a0ad_aux(rhoi, temp_aux)

(-8740.834882529642, array([0.06372576, 0.25243962]))

The ``eos.dOm_aux`` method computes the grand thermodynamic potential diffenrece and non-bonded sites fractions.

Used to compute the interfacial tension in SGT.

In [11]:
mu0 = np.array([-0.35114761, -1.0850094 ])
eos.dOm_aux(rhoi, temp_aux, mu0, P/RT)

(0.002062643585048818, array([0.06372576, 0.25243962]))

The ``eos.sgt_adim`` method computes the factor needed to make SGT system of equations consistent. 
It returns the temperature factor, pressure factor, density factor, interfacial tension factor and interfacial length factor.

In [12]:
eos.sgt_adim(T)

(1.0, 0.0003758510865637856, 1.0, 3.086918863047295e-05, 8.619041569518275)

The ``eos.ci`` method computes influence parameter matrix at given temperature.

In [13]:
eos.ci(T)

array([[3.58151059e-19, 1.37958452e-19],
       [1.37958452e-19, 5.31410809e-20]])